In [2]:
import sqlite3
import numpy as np

In [3]:
!ls

challenge-2.ipynb database.sqlite   introductory.md
challenge-3.ipynb introductory      sql-tables


**Question 1:** Which team scored the most points when playing at home?

In [4]:
conn = sqlite3.connect('database.sqlite')

In [5]:
c = conn.cursor()

In [5]:
conn

In [6]:
list(c.execute('''SELECT SUM(m.home_team_goal), t.team_long_name
                  FROM Match m
                  JOIN Team t
                  ON team_api_id = home_team_api_id
                  GROUP BY 2
                  ORDER BY 1 DESC
                  LIMIT 1;'''))

[(505, 'Real Madrid CF')]

Real Madrid CF scored the most goals at home. They scored 505 goals at home.

**Question 2:** Did this team also score the most points when playing away?

In [14]:
list(c.execute('''SELECT SUM(m.away_team_goal), t.team_long_name
                  FROM Match m
                  JOIN Team t
                  ON team_api_id = away_team_api_id
                  GROUP BY 2
                  ORDER BY 1 DESC
                  LIMIT 1;'''))

[(354, 'FC Barcelona')]

**Answer:** No. FC Barcelona scored the most points while away.

**Question 3:** How many matches resulted in a tie?

In [8]:
ties = list(c.execute('''SELECT COUNT(*)
                  FROM Match m
                  WHERE m.home_team_goal = m.away_team_goal;'''))

ties

[(6596,)]

**Answer:** 6596 matches ended in ties.

**Question 4:** How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [30]:
names = list(c.execute('''SELECT COUNT(*)
                          FROM Player pa
                          WHERE pa.player_name LIKE '%Smith' '''))

In [31]:
names_anywhere = list(c.execute('''SELECT COUNT(*)
                          FROM Player pa
                          WHERE pa.player_name LIKE '%Smith%' '''))

In [32]:
print(names)
print(names_anywhere)

[(18,)]
[(18,)]


**Answer:** 18 for both.

**Question 5:** What was the median tie score? Use the value determined in the previous question for the number of tie games. Hint: PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the WITH command to store stepwise results as a table and then operate on these results.

In [11]:
tie_scores = list(c.execute('''WITH t1 AS (SELECT COUNT(*)
                               FROM Match m
                               WHERE m.home_team_goal = m.away_team_goal),
                               
                               t2 AS (SELECT m.home_team_goal AS score
                               FROM Match m
                               WHERE m.home_team_goal = m.away_team_goal
                               ORDER BY 1 ASC
                               LIMIT (SELECT * FROM t1)/2)
                               
                               SELECT t2.score
                               FROM t2
                               ORDER BY 1 DESC 
                               LIMIT 1'''))

In [12]:
tie_scores[0][0]

1

Median tie score is 1.

**Question 6:** What percentage of players prefer their left or right foot? Hint: Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [11]:
list(c.execute('''
                SELECT 
                FROM player_attributes
               '''))

[(1,
  218353,
  505942,
  '2016-02-18 00:00:00',
  67,
  71,
  'right',
  'medium',
  'medium',
  49,
  44,
  71,
  61,
  44,
  51,
  45,
  39,
  64,
  49,
  60,
  64,
  59,
  47,
  65,
  55,
  58,
  54,
  76,
  35,
  71,
  70,
  45,
  54,
  48,
  65,
  69,
  69,
  6,
  11,
  10,
  8,
  8),
 (2,
  218353,
  505942,
  '2015-11-19 00:00:00',
  67,
  71,
  'right',
  'medium',
  'medium',
  49,
  44,
  71,
  61,
  44,
  51,
  45,
  39,
  64,
  49,
  60,
  64,
  59,
  47,
  65,
  55,
  58,
  54,
  76,
  35,
  71,
  70,
  45,
  54,
  48,
  65,
  69,
  69,
  6,
  11,
  10,
  8,
  8),
 (3,
  218353,
  505942,
  '2015-09-21 00:00:00',
  62,
  66,
  'right',
  'medium',
  'medium',
  49,
  44,
  71,
  61,
  44,
  51,
  45,
  39,
  64,
  49,
  60,
  64,
  59,
  47,
  65,
  55,
  58,
  54,
  76,
  35,
  63,
  41,
  45,
  54,
  48,
  65,
  66,
  69,
  6,
  11,
  10,
  8,
  8),
 (4,
  218353,
  505942,
  '2015-03-20 00:00:00',
  61,
  65,
  'right',
  'medium',
  'medium',
  48,
  43,
  70,
  60,


In [18]:
list(c.execute('''
                  WITH t1 AS (SELECT DISTINCT player_fifa_api_id, CASE preferred_foot WHEN 'right' THEN 1
                              WHEN 'left' THEN 0
                              END AS foot
                              FROM player_attributes
                              WHERE preferred_foot IS NOT NULL)
                    
                  SELECT AVG(t1.foot)
                  FROM t1 
                '''))

[(0.7371315983909367,)]

73.7% of players are right footed.